In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
#import dash_leaflet as dl
from dash import Dash, dcc, html
import plotly.express as px
# from dash import dash_table
from dash.dependencies import Input, Output, State
# import base64
# JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
# import os

# Configure the plotting routine
# import numpy as np

import pandas as pd
from dash import dash_table
# import dash_table

#import matplotlib.pyplot as plt

from CRUD_Python_Module import AnimalShelterCRUD


USERNAME = "aacuser"
PASSWORD = "ab@0ne"

crud = AnimalShelterCRUD(USERNAME, PASSWORD)
print("CRUD connected!")

docs = crud.read({})

df = pd.DataFrame(docs)

if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)
    
df.head()

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

app.layout = html.Div([
# html.Div(id='hidden-div', style={'display':'none'}),
# Header row: Logo + title + my name identifier
    
html.H3("Rescue Type Filter"),

dcc.RadioItems(
    id="filter-type",
    
    options=[
     {"label": "Reset (All)", "value": "Reset"},
        {"label": "Water Rescue", "value": "Water Rescue"},
        {"label": "Mountain or Wilderness Rescue", "value": "Mountain/Wilderness Rescue"},
        {"label": "Disaster Rescue or Individual Tracking", "value": "Disaster/Tracking"},
    ],    
    value="Reset",
    inline=True
),

html.Br(),

    html.Div([
        html.Img(
            src=app.get_asset_url("GraziosoSalvareLogo.png"),
            style={"height": "80px", "marginRight": "20px"}
        ),
                 
        html.Div([
            html.H2("Grazioso Salvare - Rescue Dog Dashboard"),
            html.H4("Developed by Beverly Vaughan")
        ])
    ], style={"display": "flex", "alignItems": "center"}),
        
    html.Hr(),
        
    html.H3("Unfiltered Austin Animal Center Outcomes Data"),
        
    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict("records"),
        
        # Helpful usability features
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        
        style_table={"overflowX": "auto"},
        style_cell={"textAlign": "left", "padding": "6px"},
        
    )
])
        
# Age requirement: 2 years or less (weeks)
MAX_WEEKS = 104

# Specs: use age_upon_outcome_in_weeks
WATER_BREEDS = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]
MOUNTAIN_BREEDS = ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler"]
DISASTER_BREEDS = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]

def build_query(rescue_type: str) -> dict:
    # Reset shows everything (unfiltered)
    if rescue_type == "Reset":
        return {}

    if rescue_type == "Water Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": WATER_BREEDS},
            "sex_upon_outcome": "Intact Female",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }

    if rescue_type == "Mountain/Wilderness Rescue":
        return {
            "animal_type": "Dog",
            "breed": {"$in": MOUNTAIN_BREEDS},
            "sex_upon_outcome": "Intact Male",
            "age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156},
        }

    # Disaster or Individual Tracking
    return {
        "animal_type": "Dog",
        "breed": {"$in": DISASTER_BREEDS},
        "sex_upon_outcome": "Intact Male",
        "age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300},
    }




filter_options = ["Reset (All)", "Water Rescue", "Mountain/Wilderness Rescue", "Disaster/Tracking"]

@app.callback(
    Output("datatable-id", "data"),
    Input("filter-type", "value")
)
def update_table(selected_filter):
    query = build_query(selected_filter)
    docs = crud.read(query)

    df_filtered = pd.DataFrame(docs)
    if len(df_filtered) == 0:
        return []

    if "_id" in df_filtered.columns:
        df_filtered.drop(columns=["_id"], inplace=True)

    return df_filtered.to_dict("records")

# Run app and display result in jupyb mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server(host="0.0.0.0", port=8051, debug=True, use_reloader=False)


CRUD connected!
Dash app running on http://0.0.0.0:8051/
